# 0 - Load the modules

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.DoubleType
import org.apache.spark.sql.functions._ // ajout hbc 9 oct 2020, nécessaire pour le split et col
import org.apache.spark.sql.functions.rand


/* ---------------------------------------------------  
    Import Spark ML modules
-------------- -------------------------------------- */
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel, Tokenizer}
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer, VectorAssembler} //, OneHotEncoderEstimator}

import org.apache.spark.ml.feature.OneHotEncoder // hbc le 9 oct 2020 : OneHotEncoderEstimator est une version trop récente

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.PipelineModel

import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier, LogisticRegression, LogisticRegressionModel, GBTClassificationModel, GBTClassifier}

import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}

import org.apache.spark.ml.tuning.{CrossValidator, TrainValidationSplit, ParamGridBuilder,CrossValidatorModel}

import org.apache.spark.ml.param.ParamMap

Intitializing Scala interpreter ...

Spark Web UI available at http://0d06e632b0ca:4042
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1621270407163)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.DoubleType
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.rand
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel, Tokenizer}
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer, VectorAssembler}
import org.apache.spark.ml.feature.OneHotEncoder
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.PipelineModel
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier, LogisticRegression, LogisticRegressionModel, GBTClassificationModel, GBTClassifier}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.tu...


# 1 - Start a Spark session

In [ ]:


val spark = SparkSession
.builder
.appName("flight")
.config("spark.master", "local")
.getOrCreate()

import spark.implicits._



# 3 - Load the data from parquet file and preprocess

## 3.1 - Load the data from parquet

In [3]:
// COMMAND ----------

// Lecture parquet et conversion de Th du boolean vers string

//val workingDir = "/user/user388/dir_flight_hdfs/"
//val workingDir = ""
val workingDir = "data_parquet/"

val JT  = spark.read.parquet(workingDir + "flight_1M.parquet") // 1 mois de données = jan 2012 // V2 "test_outputFull_1M_V2.parquet""
.withColumn("Th",$"Th".cast("string")).limit(1000)
JT.show(5)

+------+----+-------------+-------------+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+----------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------

workingDir: String = data/
JT: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ORIGIN: string, DEST: string ... 69 more fields]


## 3.2 - Balance the data to get same proportion for delaid and on time flights

In [4]:
// on compte le nbre total de lignes dans nos données
val nb_total = JT.count

// on compte le nbre de label en retard (ie 1.0)
val nb_delay = JT.filter(JT("Th") === "true").count.toInt

// on compte le nbre de label à l'heure (ie 0.0)
val nb_onTime = JT.filter(JT("Th") === "false").count.toInt

// on prend le plus petit nbre
val nb_selection = Math.min(nb_onTime, nb_delay)

// on mélange nos données
val JT_ok_shuffled  = JT.orderBy(rand())

// on prend autant en retard que on time
val JT_ok_shuffled_delay=JT_ok_shuffled.filter(JT_ok_shuffled("Th") === "true").limit(nb_selection)
val JT_ok_shuffled_onTime=JT_ok_shuffled.filter(JT_ok_shuffled("Th") === "false").limit(nb_selection)
val JT_balanced = JT_ok_shuffled_delay.unionAll(JT_ok_shuffled_onTime)

nb_total: Long = 1000
nb_delay: Int = 502
nb_onTime: Int = 498
nb_selection: Int = 498
JT_ok_shuffled: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ORIGIN: string, DEST: string ... 69 more fields]
JT_ok_shuffled_delay: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ORIGIN: string, DEST: string ... 69 more fields]
JT_ok_shuffled_onTime: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ORIGIN: string, DEST: string ... 69 more fields]
JT_balanced: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ORIGIN: string, DEST: string ... 69 more fields]


# 3.3 - Convert weather data array to string

In [5]:
val convertToString = udf((array: Seq[String]) => {array.mkString(",")})


val JT_flatt = JT_balanced.na.drop()

    .withColumn("slot1_D", convertToString($"slot1_D"))
    .withColumn("slot2_D", convertToString($"slot2_D"))
    .withColumn("slot3_D", convertToString($"slot3_D"))
    .withColumn("slot4_D", convertToString($"slot4_D"))
    .withColumn("slot5_D", convertToString($"slot5_D"))
    .withColumn("slot6_D", convertToString($"slot6_D"))
    .withColumn("slot7_D", convertToString($"slot7_D"))
    .withColumn("slot8_D", convertToString($"slot8_D"))
    .withColumn("slot9_D", convertToString($"slot9_D"))
    .withColumn("slot10_D", convertToString($"slot10_D"))
    .withColumn("slot11_D", convertToString($"slot11_D"))
    .withColumn("slot12_D", convertToString($"slot12_D"))

    .withColumn("slot1_A", convertToString($"slot1_A"))
    .withColumn("slot2_A", convertToString($"slot2_A"))
    .withColumn("slot3_A", convertToString($"slot3_A"))
    .withColumn("slot4_A", convertToString($"slot4_A"))
    .withColumn("slot5_A", convertToString($"slot5_A"))
    .withColumn("slot6_A", convertToString($"slot6_A"))
    .withColumn("slot7_A", convertToString($"slot7_A"))
    .withColumn("slot8_A", convertToString($"slot8_A"))
    .withColumn("slot9_A", convertToString($"slot9_A"))
    .withColumn("slot10_A", convertToString($"slot10_A"))
    .withColumn("slot11_A", convertToString($"slot11_A"))
    .withColumn("slot12_A", convertToString($"slot12_A"))

    .select("Th", "ORIGIN", "DEST",
            "slot1_D", "slot2_D", "slot3_D", "slot4_D", "slot5_D", "slot6_D", "slot7_D", "slot8_D", "slot9_D", "slot10_D", "slot11_D", "slot12_D",
            "slot1_A", "slot2_A", "slot3_A", "slot4_A", "slot5_A", "slot6_A", "slot7_A", "slot8_A", "slot9_A", "slot10_A", "slot11_A", "slot12_A")



convertToString: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4206/0x000000084167a840@4683592c,StringType,List(Some(class[value[0]: array<string>])),Some(class[value[0]: string]),None,true,true)
JT_flatt: org.apache.spark.sql.DataFrame = [Th: string, ORIGIN: string ... 25 more fields]


## 3.4 - Explode weather condition array to build new columns
(for departure and arrival and the selected time slot)

## Drop unused time slots

In [6]:
val JT_flat = JT_flatt
    // niveau 1 du getItem
    
    .withColumn("S1D_Time",split(col("slot1_D"), ",").getItem(0))
    .withColumn("S1D_H",split(col("slot1_D"), ",").getItem(1).cast(DoubleType))
    .withColumn("S1D_Wd",split(col("slot1_D"), ",").getItem(2).cast(DoubleType))
    .withColumn("S1D_Ws",split(col("slot1_D"), ",").getItem(3).cast(DoubleType))
    .withColumn("S1D_P",split(col("slot1_D"), ",").getItem(4).cast(DoubleType))
    .withColumn("S1D_T6",split(col("slot1_D"), ",").getItem(5).cast(DoubleType))
    .withColumn("S1D_S",split(col("slot1_D"), ",").getItem(6))
    .withColumn("S1D_D",split(col("slot1_D"), ",").getItem(7))
    .withColumn("S1D_V",split(col("slot1_D"), ",").getItem(8).cast(DoubleType))

    .withColumn("S6D_Time", split(col("slot6_D"), ",").getItem(0))
    .withColumn("S6D_H", split(col("slot6_D"), ",").getItem(1).cast(DoubleType))
    .withColumn("S6D_Wd", split(col("slot6_D"), ",").getItem(2).cast(DoubleType))
    .withColumn("S6D_Ws", split(col("slot6_D"), ",").getItem(3).cast(DoubleType))
    .withColumn("S6D_P", split(col("slot6_D"), ",").getItem(4).cast(DoubleType))
    .withColumn("S6D_T6", split(col("slot6_D"), ",").getItem(5).cast(DoubleType))
    .withColumn("S6D_S", split(col("slot6_D"), ",").getItem(6))
    .withColumn("S6D_D", split(col("slot6_D"), ",").getItem(7))
    .withColumn("S6D_V", split(col("slot6_D"), ",").getItem(8).cast(DoubleType))

    .withColumn("S12D_Time", split(col("slot12_D"), ",").getItem(0))
    .withColumn("S12D_H", split(col("slot12_D"), ",").getItem(1).cast(DoubleType))
    .withColumn("S12D_Wd", split(col("slot12_D"), ",").getItem(2).cast(DoubleType))
    .withColumn("S12D_Ws", split(col("slot12_D"), ",").getItem(3).cast(DoubleType))
    .withColumn("S12D_P", split(col("slot12_D"), ",").getItem(4).cast(DoubleType))
    .withColumn("S12D_T6", split(col("slot12_D"), ",").getItem(5).cast(DoubleType))
    .withColumn("S12D_S", split(col("slot12_D"), ",").getItem(6))
    .withColumn("S12D_D", split(col("slot12_D"), ",").getItem(7))
    .withColumn("S12D_V", split(col("slot12_D"), ",").getItem(8).cast(DoubleType))

    .drop("slot1_D","slot2_D", "slot3_D", "slot4_D" , "slot5_D", "slot6_D", "slot7_D", "slot8_D", "slot9_D", "slot10_D", "slot11_D", "slot12_D")


    .withColumn("S1A_Time", split(col("slot1_A"), ",").getItem(0))
    .withColumn("S1A_H", split(col("slot1_A"), ",").getItem(1).cast(DoubleType))
    .withColumn("S1A_Wd", split(col("slot1_A"), ",").getItem(2).cast(DoubleType))
    .withColumn("S1A_Ws", split(col("slot1_A"), ",").getItem(3).cast(DoubleType))
    .withColumn("S1A_P", split(col("slot1_A"), ",").getItem(4).cast(DoubleType))
    .withColumn("S1A_T6", split(col("slot1_A"), ",").getItem(5).cast(DoubleType))
    .withColumn("S1A_S", split(col("slot1_A"), ",").getItem(6))
    .withColumn("S1A_D", split(col("slot1_A"), ",").getItem(7))
    .withColumn("S1A_V", split(col("slot1_A"), ",").getItem(8).cast(DoubleType))

    .withColumn("S6A_Time", split(col("slot6_A"), ",").getItem(0))
    .withColumn("S6A_H", split(col("slot6_A"), ",").getItem(1).cast(DoubleType))
    .withColumn("S6A_Wd", split(col("slot6_A"), ",").getItem(2).cast(DoubleType))
    .withColumn("S6A_Ws", split(col("slot6_A"), ",").getItem(3).cast(DoubleType))
    .withColumn("S6A_P", split(col("slot6_A"), ",").getItem(4).cast(DoubleType))
    .withColumn("S6A_T6", split(col("slot6_A"), ",").getItem(5).cast(DoubleType))
    .withColumn("S6A_S", split(col("slot6_A"), ",").getItem(6))
    .withColumn("S6A_D", split(col("slot6_A"), ",").getItem(7))
    .withColumn("S6A_V", split(col("slot6_A"), ",").getItem(8).cast(DoubleType))

    .withColumn("S12A_Time", split(col("slot12_A"), ",").getItem(0))
    .withColumn("S12A_H", split(col("slot12_A"), ",").getItem(1).cast(DoubleType))
    .withColumn("S12A_Wd", split(col("slot12_A"), ",").getItem(2).cast(DoubleType))
    .withColumn("S12A_Ws", split(col("slot12_A"), ",").getItem(3).cast(DoubleType))
    .withColumn("S12A_P", split(col("slot12_A"), ",").getItem(4).cast(DoubleType))
    .withColumn("S12A_T6", split(col("slot12_A"), ",").getItem(5).cast(DoubleType))
    .withColumn("S12A_S", split(col("slot12_A"), ",").getItem(6))
    .withColumn("S12A_D", split(col("slot12_A"), ",").getItem(7))
    .withColumn("S12A_V", split(col("slot12_A"), ",").getItem(8).cast(DoubleType))

    .drop("slot1_A", "slot2_A", "slot3_A","slot4_A" , "slot5_A", "slot6_A", "slot7_A", "slot8_A", "slot9_A", "slot10_A", "slot11_A", "slot12_A")

JT_flat: org.apache.spark.sql.DataFrame = [Th: string, ORIGIN: string ... 55 more fields]


# 4 - Features indexation, Target indexation, Training/Test Split

In [7]:
/* ------------------------------------------  
    Target indexation with StringIndexer
-------------- ---------------------------- */
val labName = "Th"
val labelIndexer = new StringIndexer()
    .setInputCol(labName)
    .setOutputCol("indexed_" + labName)
    .setHandleInvalid("keep") // ajout pour éviter l'erreur au moment du calcul d'accuracy

/* ------------------------------------------  
    Target indexation with VectorAssembler
-------------- ---------------------------- */
val featuresCols = Array("S1D_H", "S1D_Wd","S1D_Ws","S1D_P","S1D_T6","S1D_V","S6D_H", "S6D_Wd","S6D_Ws","S6D_P","S6D_T6","S6D_V", "S12D_H", "S12D_Wd","S12D_Ws","S12D_P","S12D_T6","S12D_V","S1A_H", "S1A_Wd","S1A_Ws","S1A_P","S1A_T6","S1A_V","S6A_H", "S6A_Wd","S6A_Ws","S6A_P","S6A_T6","S6A_V","S12A_H", "S12A_Wd","S12A_Ws","S12A_P","S12A_T6","S12A_V") // sans S & D
val featureIndexer = new VectorAssembler()
    .setInputCols(featuresCols)
    .setOutputCol("indexed_Features")
    .setHandleInvalid("skip")

/* ------------------------------------------  
    Training/Test random Split
-------------- ---------------------------- */
val Array(trainingData, testData) = JT_flat.randomSplit(Array(0.7, 0.3), seed = 12345)
trainingData.persist()

trainingData.printSchema()

root
 |-- Th: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- S1D_Time: string (nullable = true)
 |-- S1D_H: double (nullable = true)
 |-- S1D_Wd: double (nullable = true)
 |-- S1D_Ws: double (nullable = true)
 |-- S1D_P: double (nullable = true)
 |-- S1D_T6: double (nullable = true)
 |-- S1D_S: string (nullable = true)
 |-- S1D_D: string (nullable = true)
 |-- S1D_V: double (nullable = true)
 |-- S6D_Time: string (nullable = true)
 |-- S6D_H: double (nullable = true)
 |-- S6D_Wd: double (nullable = true)
 |-- S6D_Ws: double (nullable = true)
 |-- S6D_P: double (nullable = true)
 |-- S6D_T6: double (nullable = true)
 |-- S6D_S: string (nullable = true)
 |-- S6D_D: string (nullable = true)
 |-- S6D_V: double (nullable = true)
 |-- S12D_Time: string (nullable = true)
 |-- S12D_H: double (nullable = true)
 |-- S12D_Wd: double (nullable = true)
 |-- S12D_Ws: double (nullable = true)
 |-- S12D_P: double (nullable = true)
 |-- S12D_T6: 

labName: String = Th
labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_3c34094b16c0
featuresCols: Array[String] = Array(S1D_H, S1D_Wd, S1D_Ws, S1D_P, S1D_T6, S1D_V, S6D_H, S6D_Wd, S6D_Ws, S6D_P, S6D_T6, S6D_V, S12D_H, S12D_Wd, S12D_Ws, S12D_P, S12D_T6, S12D_V, S1A_H, S1A_Wd, S1A_Ws, S1A_P, S1A_T6, S1A_V, S6A_H, S6A_Wd, S6A_Ws, S6A_P, S6A_T6, S6A_V, S12A_H, S12A_Wd, S12A_Ws, S12A_P, S12A_T6, S12A_V)
featureIndexer: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_22ebf5da6b03, handleInvalid=skip, numInputCols=36
trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Th: string, ORIGIN: string ... 55 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Th: string, ORIGIN: string ... 55 more fields]


## Helper function for performance

In [9]:
def performance(model : org.apache.spark.ml.tuning.CrossValidatorModel,model_name : String): Any = {

            val training_predictions = model.transform(trainingData)
                                  .select("prediction", "indexed_Th")
                                  .cache()

            val test_predictions = model.transform(testData)
                                  .select("prediction", "indexed_Th")
                                   .cache()
    
            val predictions = Array(training_predictions, test_predictions)
            val names = Array(" Training ", " Test ")
    
    println(s"\n##################### ${model_name} Performance #########################")
    
    for (i <- 0 until predictions.length)
    {        
         var data = predictions(i)
         var name = names(i)
    
            // Select (prediction, true label) and compute test error.
            var evaluator = new MulticlassClassificationEvaluator()
              .setLabelCol("indexed_Th")
              .setPredictionCol("prediction")
              .setMetricName("accuracy")
              //.setMetricName("f1")

            var accuracy = evaluator.evaluate(data)


    
            evaluator = new MulticlassClassificationEvaluator()
              .setLabelCol("indexed_Th")
              .setPredictionCol("prediction")
              .setMetricName("f1")

            var f1 = evaluator.evaluate(data)
            
            
            println(s"\n---------------------- ${name} performance metrics----------------------\n")
            println(s"\t- Accuracy = ${(accuracy * 100)}")
            println(s"\t- Error = ${(1.0 - accuracy)*100}")
            println(s"\t- F1 score = ${(f1)}")
    }
      
                }

performance: (model: org.apache.spark.ml.tuning.CrossValidatorModel, model_name: String)Any


# 5 - Random Forest : build pipeline, train and evaluate

In [11]:
//########## instanciate Model
val rf = new RandomForestClassifier()
  .setLabelCol("indexed_Th")
  .setFeaturesCol("indexed_Features")

//########## define pipeline steps
val steps_rf = Array(labelIndexer, featureIndexer, rf)

//########## create pipeline with steps and model
val pipeline_rf = new Pipeline().setStages(steps_rf)

//########## Set grid search
val paramGrid_rf = new ParamGridBuilder()
  //.addGrid(vectorIndexer.maxCategories, Array(18))
  .addGrid(rf.maxBins, Array(200))
  .addGrid(rf.maxDepth, Array(20))
  .addGrid(rf.numTrees, Array(200))
  .addGrid(rf.impurity, Array("entropy"))//, "gini"))
  .build()

//########## Set evaluator for crossvalidation
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexed_Th")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

//########## Set 3 folds crossvalidator with pipeline and 
val crossval_rf = new CrossValidator()
    .setEstimator(pipeline_rf)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid_rf)
    .setNumFolds(3)

//########## Fit the model
val modelRF_Grid = crossval_rf.fit(trainingData)

//########## Print model best parameters
println("######################## Model best parameters ########################")
println(modelRF_Grid.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap)

{
	rfc_769ca594add6-bootstrap: true,
	rfc_769ca594add6-cacheNodeIds: false,
	rfc_769ca594add6-checkpointInterval: 10,
	rfc_769ca594add6-featureSubsetStrategy: auto,
	rfc_769ca594add6-featuresCol: indexed_Features,
	rfc_769ca594add6-impurity: entropy,
	rfc_769ca594add6-labelCol: indexed_Th,
	rfc_769ca594add6-leafCol: ,
	rfc_769ca594add6-maxBins: 200,
	rfc_769ca594add6-maxDepth: 20,
	rfc_769ca594add6-maxMemoryInMB: 256,
	rfc_769ca594add6-minInfoGain: 0.0,
	rfc_769ca594add6-minInstancesPerNode: 1,
	rfc_769ca594add6-minWeightFractionPerNode: 0.0,
	rfc_769ca594add6-numTrees: 200,
	rfc_769ca594add6-predictionCol: prediction,
	rfc_769ca594add6-probabilityCol: probability,
	rfc_769ca594add6-rawPredictionCol: rawPrediction,
	rfc_769ca594add6-seed: 207336481,
	rfc_769ca594add6-subsamplingRate: 1.0
}


rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_769ca594add6
steps_rf: Array[org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable}}] = Array(strIdx_3c34094b16c0, VectorAssembler: uid=vecAssembler_22ebf5da6b03, handleInvalid=skip, numInputCols=36, rfc_769ca594add6)
pipeline_rf: org.apache.spark.ml.Pipeline = pipeline_9c7bd8212e59
paramGrid_rf: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_769ca594add6-impurity: entropy,
	rfc_769ca594add6-maxBins: 200,
...


In [10]:
performance(modelRF_Grid,"Random Forest")


##################### Random Forest Performance #########################

----------------------  Training  performance metrics----------------------

	- Accuracy = 100.0
	- Error = 0.0
	- F1 score = 1.0

----------------------  Test  performance metrics----------------------

	- Accuracy = 76.26459143968872
	- Error = 23.73540856031129
	- F1 score = 0.7628620018733485


res2: Any = ()


In [195]:
/*import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics*/

# 6 - GBT : build pipeline, train and evaluate

In [12]:
//########## instanciate Model
val gbt = new GBTClassifier()
  .setLabelCol("indexed_Th")
  .setFeaturesCol("indexed_Features")
  .setFeatureSubsetStrategy("auto")

//########## define pipeline steps
val steps_gbt = Array(labelIndexer, featureIndexer, gbt)

//########## create pipeline with steps and model
val pipeline_gbt = new Pipeline().setStages(steps_gbt)

//########## Set grid search
val paramGrid_gbt = new ParamGridBuilder()
  //.addGrid(vectorIndexer.maxCategories, Array(18))
  .addGrid(gbt.maxBins, Array(200))
  .addGrid(gbt.maxIter, Array(50))
  .addGrid(gbt.maxDepth, Array(20))
  //.addGrid(gbt.numTrees, Array(25))
  //.addGrid(gbt.impurity, Array("gini"))//, "gini"))
  .build()

//########## Set 3 folds crossvalidator with pipeline and 
val crossval = new CrossValidator()
    .setEstimator(pipeline_gbt)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid_gbt)
    .setNumFolds(3)

//########## Fit the model
val model_gbt_Grid = crossval.fit(trainingData)

//########## Print model best parameters
println("######################## Model best parameters ########################")
println(model_gbt_Grid.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap)

gbt: org.apache.spark.ml.classification.GBTClassifier = gbtc_c19788c54ac3
steps_gbt: Array[org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable}}] = Array(strIdx_3c34094b16c0, VectorAssembler: uid=vecAssembler_22ebf5da6b03, handleInvalid=skip, numInputCols=36, gbtc_c19788c54ac3)
pipeline_gbt: org.apache.spark.ml.Pipeline = pipeline_64e3de9216b2
paramGrid_gbt: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	gbtc_c19788c54ac3-maxBins: 200,
	gbtc_c19788c54ac3-maxDepth: 20,
	gbtc_...


In [13]:
performance(model_gbt_Grid,"Gradient Boosted Tree")


##################### Gradient Boosted Tree Performance #########################

----------------------  Training  performance metrics----------------------

	- Accuracy = 100.0
	- Error = 0.0
	- F1 score = 1.0

----------------------  Test  performance metrics----------------------

	- Accuracy = 67.31517509727627
	- Error = 32.68482490272373
	- F1 score = 0.6734097368402401


res4: Any = ()


# 7 - Logistic Regression : build pipeline, train and evaluate

In [14]:
//########## instanciate Model
val lr = new LogisticRegression()
  .setLabelCol("indexed_Th")
  .setFeaturesCol("indexed_Features")

//########## define pipeline steps
val steps_lr = Array(labelIndexer, featureIndexer, lr)

//########## create pipeline with steps and model
val pipeline_lr = new Pipeline().setStages(steps_lr)

//########## Set grid search
val paramGrid_lr = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.3))
  .addGrid(lr.maxIter, Array(50))
  .addGrid(lr.elasticNetParam, Array(0.8))
  .build()

//########## Set 3 folds crossvalidator with pipeline and 
val crossval = new CrossValidator()
    .setEstimator(pipeline_lr)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid_lr)
    .setNumFolds(3)


//########## Fit the model
val model_lr_Grid = crossval.fit(trainingData)

//########## Print model best parameters
println("######################## Model best parameters ########################")
println(model_lr_Grid.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap)

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_28a8c5f32d7c
steps_lr: Array[org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable}}] = Array(strIdx_3c34094b16c0, VectorAssembler: uid=vecAssembler_22ebf5da6b03, handleInvalid=skip, numInputCols=36, logreg_28a8c5f32d7c)
pipeline_lr: org.apache.spark.ml.Pipeline = pipeline_aaca8cfda3d9
paramGrid_lr: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_28a8c5f32d7c-elasticNetParam: 0.8,
	logreg_28a8c5f32d7c-max...


In [15]:
performance(model_lr_Grid,"Logistic Regression")


##################### Logistic Regression Performance #########################

----------------------  Training  performance metrics----------------------

	- Accuracy = 51.64319248826291
	- Error = 48.35680751173709
	- F1 score = 0.35174929868166693

----------------------  Test  performance metrics----------------------

	- Accuracy = 46.69260700389105
	- Error = 53.30739299610895
	- F1 score = 0.29724736554201203


res5: Any = ()


# 8 - Grid Search on Random Forest (the best model)

In [16]:
//########## Set grid search
val paramGrid_rf = new ParamGridBuilder()
  //.addGrid(vectorIndexer.maxCategories, Array(18))
  .addGrid(rf.maxBins, Array(100,150,200,250))
  .addGrid(rf.maxDepth, Array(10,20,30))
  .addGrid(rf.numTrees, Array(100,150,200))
  .addGrid(rf.impurity, Array("entropy","gini"))
  .build()

//########## Set evaluator for crossvalidation
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexed_Th")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

//########## Set 3 folds crossvalidator with pipeline and 
val crossval_rf = new CrossValidator()
    .setEstimator(pipeline_rf)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid_rf)
    .setNumFolds(3)

//########## Fit the model
val model = crossval_rf.fit(trainingData)

//########## Print model best parameters
println("######################## Model best parameters ########################")
println(model.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap)

######################## Model best parameters ########################
{
	rfc_769ca594add6-bootstrap: true,
	rfc_769ca594add6-cacheNodeIds: false,
	rfc_769ca594add6-checkpointInterval: 10,
	rfc_769ca594add6-featureSubsetStrategy: auto,
	rfc_769ca594add6-featuresCol: indexed_Features,
	rfc_769ca594add6-impurity: gini,
	rfc_769ca594add6-labelCol: indexed_Th,
	rfc_769ca594add6-leafCol: ,
	rfc_769ca594add6-maxBins: 100,
	rfc_769ca594add6-maxDepth: 10,
	rfc_769ca594add6-maxMemoryInMB: 256,
	rfc_769ca594add6-minInfoGain: 0.0,
	rfc_769ca594add6-minInstancesPerNode: 1,
	rfc_769ca594add6-minWeightFractionPerNode: 0.0,
	rfc_769ca594add6-numTrees: 200,
	rfc_769ca594add6-predictionCol: prediction,
	rfc_769ca594add6-probabilityCol: probability,
	rfc_769ca594add6-rawPredictionCol: rawPrediction,
	rfc_769ca594add6-seed: 207336481,
	rfc_769ca594add6-subsamplingRate: 1.0
}


paramGrid_rf: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_769ca594add6-impurity: entropy,
	rfc_769ca594add6-maxBins: 100,
	rfc_769ca594add6-maxDepth: 10,
	rfc_769ca594add6-numTrees: 100
}, {
	rfc_769ca594add6-impurity: entropy,
	rfc_769ca594add6-maxBins: 150,
	rfc_769ca594add6-maxDepth: 10,
	rfc_769ca594add6-numTrees: 100
}, {
	rfc_769ca594add6-impurity: entropy,
	rfc_769ca594add6-maxBins: 200,
	rfc_769ca594add6-maxDepth: 10,
	rfc_769ca594add6-numTrees: 100
}, {
	rfc_769ca594add6-impurity: gini,
	rfc_769ca594add6-maxBins: 100,
	rfc_769ca594add6-maxDepth: 10,
	rfc_769ca594add6-numTrees: 100
}, {
	rfc_769ca594add6-impurity: gini,
	rfc_769ca594add6-maxBins: 150,
	rfc_769ca594add6-maxDepth: 10,
	rfc_769ca594add6-numTrees: 100
}, {
	rfc_769ca594add6-impurity: gini,
	rfc_769ca594...


In [ ]:
performance(model,"Random Forest")